# Partie 1

## Modules

In [1]:
import openpyxl
from read_input import read_ressources,read_ressources_unavailabilities,read_tasks,read_tasks_unavailabilities

from gurobipy import *
import math
import numpy 

## Lecture des données

In [2]:
ressources = read_ressources(1)
ressources_unavailabilities = read_ressources_unavailabilities(1)
tasks = read_tasks(1)
tasks_unavailabilities = read_tasks_unavailabilities(1)

## Structures de données

In [3]:
def dist(lat1, long1, lat2, long2):
    if lat1==lat2 and long1==long2:
        return 0
    return(numpy.arccos(numpy.sin(2*numpy.pi/360*lat1)*numpy.sin(2*numpy.pi/360*lat2)+numpy.cos(2*numpy.pi/360*lat1)*numpy.cos(2*numpy.pi/360*lat2)*numpy.cos(2*numpy.pi/360*(long2-long1)))*6371)

#print(dist(49.18, -0.36, 49.33, -0.454))
#ACOS(SIN(lat1)*SIN(lat2)+COS(lat1)*COS(lat2)*COS(lon2-lon1))*6371
v=5/6

L_ressources = [j for j in ressources]
L_tasks = [i for i in tasks]
number_of_ressources = len(L_ressources)
number_of_tasks = len(L_tasks)
dict_pos = {task : (task.latitude, task.longitude) for task in tasks.values()}

dict_pos = {task : (tasks[L_tasks[task]].latitude, tasks[L_tasks[task]].longitude) for task in range(number_of_tasks)}
#graph = {(task1,task2): dist(float(tasks[L_tasks[task1].latitude]), float(tasks[L_tasks[task1].longitude]), float(tasks[L_tasks[task2].latitude]), float(tasks[L_tasks[task2].longitude])) for task1 in range(number_of_tasks) for task2 in range(number_of_tasks)}
print(dict_pos)

#distance[a][b] donne la distance de la tâche a vers la tâche b
distances = [[dist(tasks[L_tasks[a]].latitude,tasks[L_tasks[a]].longitude, tasks[L_tasks[b]].latitude, tasks[L_tasks[b]].longitude) for b in range(number_of_tasks)] for a in range(number_of_tasks)]

#distance_m[j][i] donne la distance de la maison de j la tâche i
distances_m = [[dist(ressources[L_ressources[j]].latitude,ressources[L_ressources[j]].longitude, tasks[L_tasks[i]].latitude, tasks[L_tasks[j]].longitude) for i in range(number_of_tasks)] for j in range(number_of_ressources)]
print(distances)
# creer locations
# creer dists

# https://gurobi.github.io/modeling-examples/technician_routing_scheduling/technician_routing_scheduling.html

{0: (44.556549383420084, -0.31939224223757195), 1: (44.967500952177986, -0.6086852638150881), 2: (45.14421541464031, -0.7342570469020379), 3: (45.264808304867096, -0.7717887212411139), 4: (45.044422793402624, -0.6687606009488057), 5: (45.19957452440505, -0.7462077931750715), 6: (45.397697776585, -0.9668192708194538), 7: (45.023479086796385, -0.8072126299796225), 8: (45.29291368453335, -0.9365361007032235), 9: (45.08146166752168, -0.8062453230620741)}
[[0, 51.08596249221184, 73.07241327115473, 86.43755876901946, 60.850457134509504, 79.01498988237664, 106.49564227726222, 64.63447114477614, 95.2100415497297, 69.86643262714132], [51.08596249221184, 0, 21.98645628823513, 35.449908003389794, 9.770684991977157, 27.973146924910353, 55.462667459489445, 16.805962863866643, 44.39332818955565, 20.04142298408054], [73.07241327115473, 21.98645628823513, 0, 13.72792654679316, 12.229619936463664, 6.226531366120405, 33.55065053552012, 14.596097786755063, 22.90006639021057, 8.97790170719549], [86.437558

## Variables de décision

In [4]:
### Create Model
m = Model("trs0")

M = 1440

### Decision variables
# Task-ressource assignment
x = {(t,r) : m.addVar(vtype = GRB.BINARY, name = f'x_{t}_{r}') for r in range(number_of_ressources) for t in range(number_of_tasks)}

# Time-task assignment
s = {t : m.addVar(vtype = GRB.INTEGER, lb = 0, ub = 1440, name = f's_{t}') for t in range(number_of_tasks)}
# Les s_t valent entre 0 et 1440 car les contraintes de temps (entre 8h et 18h) sont contenues dans les jeux de données.

# Est-ce que le travailleur se déplace entre les points a et b ?
y = {(r,a,b) : m.addVar(vtype = GRB.BINARY, name = f'y_{r}_{a}_{b}') for r in range(number_of_ressources) for a in range(number_of_tasks) for b in range(number_of_tasks)}# if a != b}
# Technician cannot leave or return to a depot that is not its base

# Is the worker used ?
u = {r: m.addVar(vtype = GRB.BINARY, name = f'u_{r}') for r in range(number_of_ressources)}

# starting task of worker
d = {(r,t) : m.addVar(vtype = GRB.BINARY, name = f'd_{r}_{t}') for r in range(number_of_ressources) for t in range(number_of_tasks)}
# ending task of worker
f = {(r,t) : m.addVar(vtype = GRB.BINARY, name = f'f_{r}_{t}') for r in range(number_of_ressources) for t in range(number_of_tasks)}

#variables de linéarisation
#H = {(r,a,b) : m.addVar(vtype = GRB.BINARY,lb = 0, ub = 1440, name = f'h_{r}_{a}_{b}') for r in range(number_of_ressources) for a in range(number_of_tasks) for b in range(number_of_tasks)}# if a != b}
#P = {(r,a,b) : m.addVar(vtype = GRB.BINARY,lb = 0, ub = 1440, name = f'p_{r}_{a}_{b}') for r in range(number_of_ressources) for a in range(number_of_tasks) for b in range(number_of_tasks)}# if a != b}


Set parameter Username
Academic license - for non-commercial use only - expires 2023-01-27


## Contraintes

In [5]:
C4_2 = [m.addConstr(y[(r,a,a)] == 0) for r in range(number_of_ressources) for a in range(number_of_tasks)]

In [6]:
nbRpT = [LinExpr() for t in range(number_of_tasks)]
for t in range(number_of_tasks):
  for r in  range(number_of_ressources):
    nbRpT[t]+=x[(t,r)]

# One ressource per task
C1 = [m.addConstr(nbRpT[t] == 1) for t in range(number_of_tasks)]

# Works only if all tasks are feasable
#nbRpTq = {t: LinExpr() for t in range(number_of_tasks)}
#for t in range(number_of_tasks):
#  for r in  range(number_of_ressources):
#    if ressources[L_ressources[r]].skill==tasks[L_tasks[t]].skill and ressources[L_ressources[r]].level>=tasks[L_tasks[t]].level:
#      nbRpTq[t]+=x[(t,r)]

# Qualified technicians
#C2 = [m.addConstr(nbRpTq[t] >= 1) for t in range(number_of_tasks)]

#C1 = [m.addConstr(quicksum(x[(t,r)] for r in range(number_of_ressources)) == 1 for t in range(number_of_tasks))]



In [7]:
nbRpTs = [LinExpr() for t in range(number_of_tasks)]
for t in range(number_of_tasks):
  for r in  range(number_of_ressources):
     nbRpTs[t]+=x[(t,r)]*(ressources[L_ressources[r]].skill!=tasks[L_tasks[r]].skill)

 # Skilled technicians
C2 = [m.addConstr(nbRpTs[t] == 0) for t in range(number_of_tasks)]

In [8]:
nbRpTl =[ LinExpr() for t in range(number_of_tasks)]
for t in range(number_of_tasks):
  for r in  range(number_of_ressources):
    nbRpTl[t]+=x[(t,r)]*(ressources[L_ressources[r]].level-tasks[L_tasks[r]].level)

# Leveled technicians
C3 = [m.addConstr(nbRpTl[t] >= 0) for t in range(number_of_tasks)]

In [9]:
# si une personne est affectée, elle arrive jusqu'à la tâche
C4 = {(i,j) : m.addConstr(quicksum(y[(j,a,i)] for a in range(number_of_tasks)) + d[(j,i)] == x[(i,j)], name = f'arrive{i}{j}') for j in range(number_of_ressources) for i in range(number_of_tasks)}

# une personne doit partir après avoir exécuté sa tâche
C5 = {(i,j) : m.addConstr(quicksum(y[(j,i,b)] for b in range(number_of_tasks)) + f[(j,i)] == x[(i,j)], name = f'part{i}{j}') for j in range(number_of_ressources) for i in range(number_of_tasks)}

In [10]:
# Non overlapping tasks

# C6 = [m.addConstr(H[(r,a,b)] >=0) for r in range(number_of_ressources) for a in range(number_of_tasks) for b in range(number_of_tasks)]# if a != b]
# C7 = [m.addConstr(H[(r,a,b)] - 1440 * y[(r,a,b)]  <=0) for r in range(number_of_ressources) for a in range(number_of_tasks) for b in range(number_of_tasks)]# if a != b]
# C8 = [m.addConstr(H[(r,a,b)] - s[a]  <=0) for r in range(number_of_ressources) for a in range(number_of_tasks) for b in range(number_of_tasks)]# if a != b]
# C9 = [m.addConstr(H[(r,a,b)] - s[a] + 1440*(1-H[(r,a,b)])  >=0) for r in range(number_of_ressources) for a in range(number_of_tasks) for b in range(number_of_tasks)]# if a != b]

# C10 = [m.addConstr(P[(r,a,b)] >=0) for r in range(number_of_ressources) for a in range(number_of_tasks) for b in range(number_of_tasks)]# if a != b]
# C11 = [m.addConstr(P[(r,a,b)] - 1440 * y[(r,a,b)]  <=0) for r in range(number_of_ressources) for a in range(number_of_tasks) for b in range(number_of_tasks)]# if a != b]
# C12 = [m.addConstr(P[(r,a,b)] - s[b]  <=0) for r in range(number_of_ressources) for a in range(number_of_tasks) for b in range(number_of_tasks)]# if a != b]
# C13 = [m.addConstr(P[(r,a,b)] - s[b] + 1440*(1-P[(r,a,b)])  >=0) for r in range(number_of_ressources) for a in range(number_of_tasks) for b in range(number_of_tasks)]# if a != b]
# C14Sum =[[LinExpr() for a in range(number_of_tasks)] for b in range(number_of_tasks)]
# for a in range(number_of_tasks):
#   for b in range(number_of_tasks):
#     for j in  range(number_of_ressources):
#       C14Sum[a][b]+=H[(j,a,b)] - P[(j,a,b)] + y[(j,a,b)] * (tasks[L_tasks[a]].duration + distances[a][b]/v)

# C14 = [m.addConstr(C14Sum[a][b] <= 0) for a in range(number_of_tasks) for b in range(number_of_tasks)]

In [11]:
# non overlapping tasks
print(distances[3][5]/v)
print(tasks[L_tasks[3]].duration)
C6 = [m.addConstr(s[b] >= s[a] + tasks[L_tasks[a]].duration + distances[a][b]/v -M*(1-quicksum(y[(j,a,b)] for j in range(number_of_ressources)))) for a in range(number_of_tasks) for b in range(number_of_tasks)]

9.030219271874246
60


In [12]:
# la premiere tache commence apres le début de la journée
#j,i=0,3
#print(ressources[L_ressources[j]].workingStart -M*(1-x[(i,j)].x) + distances_m[j][i]/v)
#M=10E2
C11 = [m.addConstr(s[i] >= ressources[L_ressources[j]].workingStart -M*(1-x[(i,j)]) + distances_m[j][i]/v) for i in range(number_of_tasks) for j in range(number_of_ressources)]

In [13]:
# la derniere tache finit avant la fin de la journee
M=10E6
C12 = [m.addConstr(ressources[L_ressources[j]].workingEnd >= s[i]+tasks[L_tasks[i]].duration + distances_m[j][i]/v -M*(1-x[(i,j)])) for i in range(number_of_tasks) for j in range(number_of_ressources)]

In [14]:
C7 = [m.addConstr(quicksum(d[(j,a)] for a in range(number_of_tasks)) == u[j]) for j in range(number_of_ressources)]
C8 = [m.addConstr(quicksum(f[(j,b)] for b in range(number_of_tasks)) == u[j]) for j in range(number_of_ressources)]

C9 = [m.addConstr(quicksum(x[(i,j)] for i in range(number_of_tasks)) <= u[j]*M) for j in range(number_of_ressources)]
C10 = [m.addConstr(quicksum(x[(i,j)] for i in range(number_of_tasks)) >= u[j]) for j in range(number_of_ressources)]

C13 = [d[(j,i)]<=x[(i,j)] for i in range(number_of_tasks) for j in range(number_of_ressources)]
C14 = [f[(j,i)]<=x[(i,j)] for i in range(number_of_tasks) for j in range(number_of_ressources)]



In [15]:
# C7 = [m.addConstr(quicksum(d[(j,a)] for a in range(number_of_tasks)) == 1) for j in range(number_of_ressources)]
# C8 = [m.addConstr(quicksum(f[(j,b)] for b in range(number_of_tasks)) == 1) for j in range(number_of_ressources)]

# # C9 = [m.addConstr(quicksum(x[(i,j)] for i in range(number_of_tasks)) <= u[j]*M) for j in range(number_of_ressources)]
# # C10 = [m.addConstr(quicksum(x[(i,j)] for i in range(number_of_tasks)) >= u[j]) for j in range(number_of_ressources)]

# C13 = [d[(j,i)]<=x[(i,j)] for i in range(number_of_tasks) for j in range(number_of_ressources)]
# C14 = [f[(j,i)]<=x[(i,j)] for i in range(number_of_tasks) for j in range(number_of_ressources)]


In [16]:
C15 = [m.addConstr(y[(j,a,b)] + y[(j,b,a)] <=1) for a in range(number_of_tasks) for b in range(number_of_tasks) for j in range(number_of_ressources)]
C16 = [m.addConstr(d[(j,a)] + f[(j,a)] <=1) for a in range(number_of_tasks) for j in range(number_of_ressources)]


## Fonction objectif et résolution

In [17]:
# -- Ajout de la fonction objectif --
couts=LinExpr()
for j in range(number_of_ressources):
  for a in range(number_of_tasks):
    for b in range(number_of_tasks):
      couts+=y[(j,a,b)]*distances[a][b]/v
m.setObjective(couts, GRB.MINIMIZE)

# -- Choix d'un paramétrage d'affichage minimaliste --
m.params.outputflag = 0 # mode muet

# -- Mise à jour du modèle  --
m.update()

# -- Affichage en mode texte du PL --
m.display()

# -- Résolution --
m.optimize()

# -- Vérification du statut et Affichage (le cas échéant) des solutions --
if m.status == GRB.INF_OR_UNBD:
    m.setParam(GRB.Param.Presolve, 0)
    m.optimize()

if m.status == GRB.INFEASIBLE:
    print(m.display(), "\n\tN'A PAS DE SOLUTION!!!")
elif m.status == GRB.UNBOUNDED:
    print(m.display(), "\n\tEST NON BORNÉ!!!")


# for j in range(number_of_tasks):
#   for i in range(number_of_ressources):
#     print(x[(j,i)].x,end=' ')
#   print("\n")

# print("\n\n\n")


# -- Affichage de la solution --
#print("La solution optimale est (d,q) = {} \navec pour valeur de l'objectif z = {}€".format((D.x, Q.x), m.objVal))
t=open("result.csv",'w')
res="taskId;performed;employeeName;startTime;\n"
for j in range(number_of_tasks):
  p=False
  for i in range(number_of_ressources):
    if round(x[(j,i)].x):
      res+="{};{};{};{};\n".format(L_tasks[j],1,L_ressources[i],int(s[j].x))
      p=True
  if not p:
    res+="{};;;;\n".format(L_tasks[j])
print(res)
t.write(res)
t.close()

print("###########################")

for a in range(number_of_tasks):
  for b in range(number_of_tasks):
    j=0
    if  round(y[(j,a,b)].x)==1:
      print("j{} - a{} - b{} - y{} // ".format(j, a,b,y[(j,a,b)].x), end="")

#7 - 4 - 2 - 1 - 7
#8 - 6 - 8

print('\n') 
print("###########################")

for a in range(number_of_tasks):
  j=0
  if  round(d[(j,a)].x)==1:
    print("j{} - a{} - d{} // ".format(j, a,d[(j,a)].x), end="")


print('\n\n')
for a in range(number_of_tasks):
  j=0
  if  round(f[(j,a)].x)==1:
    print("j{} - a{} - d{} // ".format(j, a,f[(j,a)].x), end="")

# 3 - 5 - 2 - 9 - 7 - 4 - 1 - 0
# 6-8 okok

taskId;performed;employeeName;startTime;
T1;1;Valentin;573;
T2;1;Valentin;674;
T3;1;Valentin;875;
T4;1;Valentin;970;
T5;1;Valentin;725;
T6;1;Valentin;922;
T7;1;Ambre;519;
T8;1;Valentin;778;
T9;1;Ambre;594;
T10;1;Valentin;825;

###########################
j0 - a0 - b1 - y0.9999979000051843 // j0 - a1 - b4 - y0.999997900027106 // j0 - a2 - b5 - y0.9999978000430146 // j0 - a4 - b7 - y0.999997900023804 // j0 - a5 - b3 - y0.9999978000356742 // j0 - a7 - b9 - y0.9999978000415548 // j0 - a9 - b2 - y0.9999978000299794 // 

###########################
j0 - a0 - d0.9999979009090908 // 


j0 - a3 - d0.9999978016671467 // 

In [18]:
# import folium

# lat_tot,long_tot=0,0
# for a in range(number_of_tasks):
#   lat_tot+=tasks[L_tasks[a]].latitude
#   long_tot+=tasks[L_tasks[a]].longitude
# mid_latitude = lat_tot/number_of_tasks
# mid_longitude = long_tot/number_of_tasks
# carte=folium.Map(location=[mid_latitude, mid_longitude],zoom_start=10)
# for a in range(number_of_tasks):
#   folium.Marker([tasks[L_tasks[a]].latitude, tasks[L_tasks[a]].longitude],icon=folium.Icon(color='green')).add_to(carte)
# for r in range(number_of_ressources):
#   for a in range(number_of_tasks):
#     for b in range(number_of_tasks):
#       if round(y[(r,a,b)].x) == 1:
#         line_ab = [(tasks[L_tasks[a]].latitude, tasks[L_tasks[a]].longitude),(tasks[L_tasks[b]].latitude, tasks[L_tasks[b]].longitude)]
#         folium.PolyLine(line_ab, color="red", weight=2.5, opacity=0.8).add_to(carte)
#     if round(d[(r,a)].x) == 1:
#       line_home_t = [(ressources[L_ressources[r]].latitude,ressources[L_ressources[r]].longitude), (tasks[L_tasks[a]].latitude, tasks[L_tasks[a]].longitude)]
#       folium.PolyLine(line_home_t, color="blue", weight=2.5, opacity=0.8).add_to(carte)
#     if round(f[(r,a)].x) == 1:
#       line_t_home = [(tasks[L_tasks[a]].latitude, tasks[L_tasks[a]].longitude), (ressources[L_ressources[r]].latitude,ressources[L_ressources[r]].longitude)]
#       folium.PolyLine(line_t_home, color="blue", weight=2.5, opacity=0.8).add_to(carte)

# carte.save('Cartographie/Cartev1.html')

# carte